# pipelines.rag

> A pipeline module for Retrieval Augmented Generation (RAG)

In [ ]:
# | default_exp pipelines.rag

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
from typing import Optional, Dict, List, Any
from langchain_core.documents import Document
from onprem.utils import format_string
from onprem.llm import helpers
from pydantic import BaseModel, Field

In [ ]:
# | export

DEFAULT_QA_PROMPT = """Use the following pieces of context delimited by three backticks to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

```{context}```

Question: {question}
Helpful Answer:"""

DEFAULT_ROUTER_PROMPT = """Given the following query/question, select the most appropriate category that would contain the relevant information.

Query: {question}

Available categories:
{categories}

Select the best category from the list above, or 'none' if no category is appropriate."""


class RAGPipeline:
    """
    Retrieval-Augmented Generation pipeline for answering questions based on source documents.
    """
    
    def __init__(self, llm, qa_template: str = DEFAULT_QA_PROMPT):
        """
        Initialize RAG pipeline.
        
        Args:
            llm: The language model instance (LLM object)
            qa_template: Question-answering prompt template
        """
        self.llm = llm
        self.qa_template = qa_template
    
    def semantic_search(self,
                       query: str, # search query as string
                       limit: int = 4, # number of sources to retrieve
                       score_threshold: float = 0.0, # minimum threshold for score
                       filters: Optional[Dict[str, str]] = None, # metadata filters
                       where_document = None, # filter search results based syntax of underlying store
                       folders: Optional[list] = None, # list of folders to consider
                       **kwargs) -> List[Document]:
        """
        Perform a semantic search of the vector DB.

        The `where_document` parameter varies depending on the value of `LLM.store_type`.
        If `LLM.store_type` is 'dense', then `where_document` should be a dictionary in Chroma syntax (e.g., {"$contains": "Canada"})
        to filter results.
        If `LLM.store_type` is 'sparse', then `where_document` should be a boolean search string to filter query in Lucene syntax.
        """
        import os
        store = self.llm.load_vectorstore()
        if folders:
            folders = [folders] if isinstance(folders, str) else folders
            # This is needed because only the where argument supports the $like operator
            # and Langchain does not properly forward the where parameter to Chroma
            n_candidates = store.get_size() if store.get_size() < 10000 else 10000
            results = store.semantic_search(query, 
                                            filters=filters,
                                            where_document=where_document,
                                            limit = n_candidates, **kwargs)
            # Handle path separator differences between Windows and Unix
            if os.name == 'nt':  # Windows
                # Normalize paths for case-insensitive comparison on Windows
                normalized_folders = [os.path.normpath(f).lower().replace('\\', '/') for f in folders]
                results = [d for d in results if any(os.path.normpath(d.metadata['source']).lower().replace('\\', '/').startswith(nf) for nf in normalized_folders)]
            else:
                # On Unix systems, use direct path comparison
                results = [d for d in results if any(d.metadata['source'].startswith(f) for f in folders)]
            results = results[:limit]
            
        else:
            results = store.semantic_search(query, 
                                            filters=filters,
                                            where_document=where_document,
                                            limit = limit, **kwargs)

        return [d for d in results if d.metadata['score'] >= score_threshold]

    def _retrieve_documents(self, 
                          question: str,
                          filters: Optional[Dict[str, str]] = None,
                          where_document = None,
                          folders: Optional[list] = None,
                          limit: int = 4,
                          score_threshold: float = 0.0,
                          table_k: int = 1,
                          table_score_threshold: float = 0.35) -> List[Document]:
        """
        Retrieve relevant documents from vector database.
        """
        docs = self.semantic_search(
            question, 
            filters=filters, 
            where_document=where_document, 
            folders=folders,
            limit=limit,
            score_threshold=score_threshold
        )
        
        # Add table documents if requested
        if table_k > 0:
            table_filters = filters.copy() if filters else {}
            table_filters = dict(table_filters, table=True)
            table_docs = self.semantic_search(
                f'{question} (table)', 
                filters=table_filters, 
                where_document=where_document,
                folders=folders,
                limit=table_k,
                score_threshold=table_score_threshold
            )
            if table_docs:
                docs.extend(table_docs[:limit])
        
        return docs
    
    def _generate_answer(self, question: str, context: str, **kwargs) -> str:
        """
        Generate answer using the language model.
        """
        prompt = format_string(
            self.qa_template,
            question=question,
            context=context
        )
        return self.llm.prompt(prompt, **kwargs)
    
    def ask(self,
            question: str, # question as string
            contexts: Optional[list] = None, # optional list of contexts to answer question. If None, retrieve from vectordb.
            qa_template: Optional[str] = None, # question-answering prompt template to use
            filters: Optional[Dict[str, str]] = None, # filter sources by metadata values using Chroma metadata syntax (e.g., {'table':True})
            where_document = None, # filter sources by document content (syntax varies by store type)
            folders: Optional[list] = None, # folders to search (needed because LangChain does not forward "where" parameter)
            limit: Optional[int] = None, # Number of sources to consider. If None, use `LLM.rag_num_source_docs`.
            score_threshold: Optional[float] = None, # minimum similarity score of source. If None, use `LLM.rag_score_threshold`.
            table_k: int = 1, # maximum number of tables to consider when generating answer
            table_score_threshold: float = 0.35, # minimum similarity score for table to be considered in answer
            selfask: bool = False, # If True, use an agentic Self-Ask prompting strategy.
            router = None, # Optional KVRouter instance for automatic filtering
            **kwargs) -> Dict[str, Any]:
        """
        Answer a question using RAG approach.
        
        Args:
            question: Question to answer
            contexts: Optional list of contexts. If None, retrieve from vectordb
            qa_template: Optional custom QA prompt template
            filters: Filter sources by metadata values
            where_document: Filter sources by document content
            folders: Folders to search
            limit: Number of sources to consider
            score_threshold: Minimum similarity score
            table_k: Maximum number of tables to consider
            table_score_threshold: Minimum similarity score for tables
            selfask: Use agentic Self-Ask prompting strategy
            **kwargs: Additional arguments passed to LLM.prompt
            
        Returns:
            Dictionary with keys: answer, source_documents, question
        """
        template = qa_template or self.qa_template
        limit = limit if limit is not None else self.llm.rag_num_source_docs
        score_threshold = score_threshold if score_threshold is not None else self.llm.rag_score_threshold
        
        # Apply router if provided
        if router and not filters:
            router_filters = router.route(question)
            if router_filters:
                filters = router_filters
        elif router and filters:
            # Merge router filters with existing filters
            router_filters = router.route(question)
            if router_filters:
                combined_filters = filters.copy()
                combined_filters.update(router_filters)
                filters = combined_filters
        
        if selfask and helpers.needs_followup(question, self.llm):
            return self._ask_with_decomposition(
                question, template, filters, where_document, folders,
                limit, score_threshold, table_k, table_score_threshold, **kwargs
            )
        else:
            return self._ask_direct(
                question, contexts, template, filters, where_document, folders,
                limit, score_threshold, table_k, table_score_threshold, **kwargs
            )
    
    def _ask_direct(self,
                   question: str,
                   contexts: Optional[list],
                   qa_template: str,
                   filters: Optional[Dict[str, str]],
                   where_document,
                   folders: Optional[list],
                   limit: int,
                   score_threshold: float,
                   table_k: int,
                   table_score_threshold: float,
                   **kwargs) -> Dict[str, Any]:
        """Direct RAG without decomposition."""
        if contexts is None:
            docs = self._retrieve_documents(
                question, filters, where_document, folders,
                limit, score_threshold, table_k, table_score_threshold
            )
            context = '\n\n'.join([d.page_content for d in docs])
        else:
            docs = [Document(page_content=c, metadata={'source': '<SUBANSWER>'}) for c in contexts]
            context = "\n\n".join(contexts)
        
        answer = self._generate_answer(question, context, **kwargs)
        
        return {
            'question': question,
            'answer': answer,
            'source_documents': docs
        }
    
    def _ask_with_decomposition(self,
                               question: str,
                               qa_template: str,
                               filters: Optional[Dict[str, str]],
                               where_document,
                               folders: Optional[list],
                               limit: int,
                               score_threshold: float,
                               table_k: int,
                               table_score_threshold: float,
                               **kwargs) -> Dict[str, Any]:
        """RAG with question decomposition (Self-Ask)."""
        subquestions = helpers.decompose_question(question, self.llm)
        subanswers = []
        sources = []
        
        for q in subquestions:
            res = self._ask_direct(
                q, None, qa_template, filters, where_document, folders,
                limit, score_threshold, table_k, table_score_threshold, **kwargs
            )
            subanswers.append(res['answer'])
            for doc in res['source_documents']:
                doc.metadata = dict(doc.metadata, subquestion=q)
            sources.extend(res['source_documents'])
        
        # Generate final answer based on subanswers
        res = self._ask_direct(
            question, subanswers, qa_template, filters, where_document, folders,
            limit, score_threshold, table_k, table_score_threshold, **kwargs
        )
        res['source_documents'] = sources
        
        return res




In [ ]:
show_doc(RAGPipeline.ask)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/pipelines/rag.py#L143){target="_blank" style="float:right; font-size:smaller"}

### RAGPipeline.ask

>      RAGPipeline.ask (question:str, contexts:Optional[list]=None,
>                       qa_template:Optional[str]=None,
>                       filters:Optional[Dict[str,str]]=None,
>                       where_document=None, folders:Optional[list]=None,
>                       limit:Optional[int]=None,
>                       score_threshold:Optional[float]=None, table_k:int=1,
>                       table_score_threshold:float=0.35, selfask:bool=False,
>                       router=None, **kwargs)

*Answer a question using RAG approach.

Args:
    question: Question to answer
    contexts: Optional list of contexts. If None, retrieve from vectordb
    qa_template: Optional custom QA prompt template
    filters: Filter sources by metadata values
    where_document: Filter sources by document content
    folders: Folders to search
    limit: Number of sources to consider
    score_threshold: Minimum similarity score
    table_k: Maximum number of tables to consider
    table_score_threshold: Minimum similarity score for tables
    selfask: Use agentic Self-Ask prompting strategy
    **kwargs: Additional arguments passed to LLM.prompt

Returns:
    Dictionary with keys: answer, source_documents, question*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| question | str |  | question as string |
| contexts | Optional | None | optional list of contexts to answer question. If None, retrieve from vectordb. |
| qa_template | Optional | None | question-answering prompt template to use |
| filters | Optional | None | filter sources by metadata values using Chroma metadata syntax (e.g., {'table':True}) |
| where_document | NoneType | None | filter sources by document content (syntax varies by store type) |
| folders | Optional | None | folders to search (needed because LangChain does not forward "where" parameter) |
| limit | Optional | None | Number of sources to consider. If None, use `LLM.rag_num_source_docs`. |
| score_threshold | Optional | None | minimum similarity score of source. If None, use `LLM.rag_score_threshold`. |
| table_k | int | 1 | maximum number of tables to consider when generating answer |
| table_score_threshold | float | 0.35 | minimum similarity score for table to be considered in answer |
| selfask | bool | False | If True, use an agentic Self-Ask prompting strategy. |
| router | NoneType | None | Optional KVRouter instance for automatic filtering |
| kwargs | VAR_KEYWORD |  |  |
| **Returns** | **Dict** |  |  |

In [ ]:
show_doc(RAGPipeline.semantic_search)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/pipelines/rag.py#L49){target="_blank" style="float:right; font-size:smaller"}

### RAGPipeline.semantic_search

>      RAGPipeline.semantic_search (query:str, limit:int=4,
>                                   score_threshold:float=0.0,
>                                   filters:Optional[Dict[str,str]]=None,
>                                   where_document=None,
>                                   folders:Optional[list]=None, **kwargs)

*Perform a semantic search of the vector DB.

The `where_document` parameter varies depending on the value of `LLM.store_type`.
If `LLM.store_type` is 'dense', then `where_document` should be a dictionary in Chroma syntax (e.g., {"$contains": "Canada"})
to filter results.
If `LLM.store_type` is 'sparse', then `where_document` should be a boolean search string to filter query in Lucene syntax.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| query | str |  | search query as string |
| limit | int | 4 | number of sources to retrieve |
| score_threshold | float | 0.0 | minimum threshold for score |
| filters | Optional | None | metadata filters |
| where_document | NoneType | None | filter search results based syntax of underlying store |
| folders | Optional | None | list of folders to consider |
| kwargs | VAR_KEYWORD |  |  |
| **Returns** | **List** |  |  |

In [ ]:
#|export

class CategorySelection(BaseModel):
    """Pydantic model for category selection response."""
    category: str = Field(description="Selected category value or 'none' if no appropriate category")


class KVRouter:
    """
    Key-Value Router for intelligent filtering based on query content.
    
    Uses an LLM to select the most appropriate field value for filtering
    based on the query/question content.
    """
    
    def __init__(self, 
                 field_name: str,
                 field_descriptions: Dict[str, str],
                 llm,
                 router_prompt: str = DEFAULT_ROUTER_PROMPT):
        """
        Initialize KV Router.
        
        Args:
            field_name: The metadata field name to filter on (e.g., 'folder')
            field_descriptions: Dict mapping field values to descriptions
                               (e.g., {'sotu': "Biden's State of the Union Address"})
            llm: The LLM instance to use for routing decisions
            router_prompt: Template for the routing prompt
        """
        self.field_name = field_name
        self.field_descriptions = field_descriptions
        self.llm = llm
        self.router_prompt = router_prompt
    
    def _format_categories(self) -> str:
        """Format field descriptions for the prompt."""
        categories = []
        for value, description in self.field_descriptions.items():
            categories.append(f"- {value}: {description}")
        return "\n".join(categories)
    
    def route(self, question: str) -> Optional[Dict[str, str]]:
        """
        Select the best field value for the given question.
        
        Args:
            question: The user's question/query
            
        Returns:
            Dictionary for filters parameter, or None if no appropriate category
            Example: {'folder': 'sotu'} or None
        """
        # Format the prompt
        categories_text = self._format_categories()
        prompt = format_string(
            self.router_prompt,
            question=question,
            categories=categories_text
        )
        
        # Use pydantic_prompt for structured output
        try:
            response = self.llm.pydantic_prompt(
                prompt, 
                pydantic_model=CategorySelection,
                attempt_fix=True  # Try to fix malformed responses
            )
            
            selected_category = response.category.strip().lower()
            
            # Check if it's a valid category (case-insensitive)
            valid_values = {v.lower(): v for v in self.field_descriptions.keys()}
            
            if selected_category in valid_values:
                return {self.field_name: valid_values[selected_category]}
            elif selected_category == 'none':
                return None
            else:
                # Fallback: if response doesn't match, return None
                return None
                
        except Exception as e:
            # Fallback to None if pydantic parsing fails
            return None
    
    def route_and_search(self, 
                        query: str,
                        rag_pipeline,
                        **search_kwargs) -> List[Document]:
        """
        Convenience method that routes and performs semantic search.
        
        Args:
            query: The search query
            rag_pipeline: RAGPipeline instance to search with
            **search_kwargs: Additional arguments passed to semantic_search
            
        Returns:
            List of Document objects
        """
        # Get routing decision
        route_filters = self.route(query)
        
        # Merge with existing filters if any
        existing_filters = search_kwargs.get('filters', {})
        if route_filters:
            if existing_filters:
                # Combine filters
                combined_filters = existing_filters.copy()
                combined_filters.update(route_filters)
                search_kwargs['filters'] = combined_filters
            else:
                search_kwargs['filters'] = route_filters
        
        # Perform search
        return rag_pipeline.semantic_search(query, **search_kwargs)

### Example: Using Query Routing with RAG

In this example, we use the `KVRouter` to route RAG queries to the correct set of ingested documents.

First, when we ingest documents, we assign a folder field to each document chunk. (You can also use the `text_callables` parameter to assign a field value based on text content.)

In [ ]:
# |  notest

from onprem import LLM
from onprem.pipelines import KVRouter
import tempfile


In [ ]:
#|notest

# Setup LLM and ingest with custom metadata
llm = LLM('openai/gpt-4o-mini', vectordb_path=tempfile.mkdtemp())
def set_folder(filepath):
    if 'sotu' in filepath:
        return 'sotu'
    elif 'ktrain_paper' in filepath:
        return 'ktrain'
    else:
        return 'na'
        
llm.ingest('tests/sample_data/sotu', file_callables={'folder': set_folder})
llm.ingest('tests/sample_data/ktrain_paper', file_callables={'folder': set_folder})


Creating new vectorstore at /tmp/tmpdx_zjj3j/dense
Loading documents from tests/sample_data/sotu


Processing and chunking 1 new documents: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 446.44it/s]


Split into 43 chunks of text (max. 1000 chars each for text; max. 2000 chars for tables)
Creating embeddings. May take some minutes...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.87it/s]

Ingestion complete! You can now query your documents using the LLM.ask or LLM.chat methods
Appending to existing vectorstore at /tmp/tmpdx_zjj3j/dense
Loading documents from tests/sample_data/ktrain_paper



Processing and chunking 6 new documents: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1258.04it/s]


Split into 22 chunks of text (max. 1000 chars each for text; max. 2000 chars for tables)
Creating embeddings. May take some minutes...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.19it/s]

Ingestion complete! You can now query your documents using the LLM.ask or LLM.chat methods


Next, we setup a `KVRouter` that returns the best key-value paper (in this case, a specific `folder` value) based on the question or query.  The key-value pair is then used to filter the documents appropriately when retrieving source documents for answer generation.  The router can be supplied direclty to the `ask` method so that only docuents in the appropriate folder are considered when generating answers.

In [ ]:
#|notest

# Create router
router = KVRouter(
  field_name='folder',
  field_descriptions={
      'sotu': "Biden's State of the Union Address",
      'ktrain': "Research papers about ktrain library, a toolkit for machine learning, text classification, and computer vision."
  },
  llm=llm
)

# Example of router
filter_dict = router.route('Tell me about image classification')
print()
print(filter_dict)

```json
{"category":"ktrain"}
```
{'folder': 'ktrain'}


In [ ]:
# |notest

# Use router with ask() - Method 1: Direct parameter
result = llm.ask(
  "What did Biden say about the economy?",
  router=router
)

```json
{"category":"sotu"}
```Biden stated that the previous trickle-down economic theory resulted in weaker economic growth, lower wages, larger deficits, and an increasing wealth gap. He emphasized a new economic vision focused on investing in America, educating Americans, and growing the workforce from the bottom up and the middle out, rather than from the top down. He highlighted the importance of infrastructure investment, noted the need for competition to lower costs, and mentioned the creation of manufacturing jobs and significant corporate investments in electric vehicles. He acknowledged that, despite record job growth and higher wages, many families are still struggling with inflation, and his top priority is to bring prices under control.

In [ ]:
#|notest

# Use router with RAG pipeline - Method 2: Direct on pipeline
rag_pipeline = llm.load_rag_pipeline()
result = rag_pipeline.ask(
  "How do I use ktrain for text classification?",
  router=router
)

```json
{"category": "ktrain"}
```To use ktrain for text classification, you can follow these general steps based on the provided context:

1. **Load and Preprocess Data**: Start by loading your text data. You may need to preprocess it to fit the model's requirements, which could involve tokenization and other language-specific preprocessing steps. Use the appropriate Preprocessor instance that ktrain provides for text data.

2. **Create Model**: You can create and customize your model using `tf.keras`. ktrain supports various model architectures suitable for text classification.

3. **Train the Model**: After creating the model, you will need to train it using your preprocessed data. This step involves setting hyperparameters such as learning rates and the number of epochs.

4. **Inspect the Model**: Once trained, inspect the model’s performance. You can generate classification reports to evaluate both its successes and failures.

5. **Apply the Model**: Finally, save the model and an

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()